# Example of how to make a movie of the CMG system

Import modules and configure the notebook.

In [1]:
# These modules are part of the python standard library
import time
import os # <--- important for making a movie

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_cmg

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_cmg)

<module 'ae353_cmg' from '/Users/timothybretl/Documents/courses/AE353/09 - AE353 (Spring 2021)/Website/examples/day13_cmg/ae353_cmg.py'>

Create an instance of the robot simulator. If you want to make a movie, it is important that `display=True`.

In [2]:
robot = ae353_cmg.RobotSimulator(damping=0., dt=0.001, display=True)

Apply PD control to keep the gimbal angle at a desired value:

In [3]:
class RobotController:
    def __init__(self, dt=0.001, q_2_des=0.):
        self.dt = dt
        
        # desired gimbal angle
        self.q_2_des = q_2_des
        
        # PD gains
        self.kp = 5.
        self.kd = 1.
    
    def run(self, q_1, v_1, q_2, v_2, q_3, v_3):
        tau_2 = - self.kp * (q_2 - self.q_2_des) - self.kd * (v_2 - 0.)
        tau_3 = 0.
        return tau_2, tau_3

controller = RobotController(dt=robot.dt, q_2_des=1.)

Run the simulation. It is a loop. At each iteration, we:
* get sensor measurements
* choose actuator commands
* go forward one time step

We also log data so that we can plot it later, if we want.

Since we are making a movie, we save an image at each time step. **This takes a long time!**

In [4]:
# Restore the simulation to its initial state
robot.reset(rotor_rpm=100.)

# Choose how long we want to run the simulation, and
# compute the corresponding number of time steps
run_time = 0.5
num_steps = int(run_time/robot.dt)

# Create a dictionary in which to store results
data = {
    't': np.empty(num_steps, dtype=float),
    'q_1': np.empty(num_steps, dtype=float),
    'v_1': np.empty(num_steps, dtype=float),
    'q_2': np.empty(num_steps, dtype=float),
    'v_2': np.empty(num_steps, dtype=float),
    'q_3': np.empty(num_steps, dtype=float),
    'v_3': np.empty(num_steps, dtype=float),
    'tau_2': np.empty(num_steps, dtype=float),
    'tau_3': np.empty(num_steps, dtype=float),
}

# Create directory to store images
os.mkdir('images')

# Save first image
robot.snapshot(os.path.join('images', f'image{0:06d}.png'))

# Run the simulation loop
start_time = time.time()
for step in range(num_steps):
    # Get the current time
    t = robot.dt * step
    
    # Get the sensor measurements
    q_1, v_1, q_2, v_2, q_3, v_3 = robot.get_sensor_measurements()
    
    # Choose the actuator command (by running the controller)
    tau_2, tau_3 = controller.run(q_1, v_1, q_2, v_2, q_3, v_3)
    
    # Log the data from this time step
    data['t'][step] = t
    data['q_1'][step] = q_1
    data['v_1'][step] = v_1
    data['q_2'][step] = q_2
    data['v_2'][step] = v_2
    data['q_3'][step] = q_3
    data['v_3'][step] = v_3
    data['tau_2'][step] = tau_2
    data['tau_3'][step] = tau_3
    
    # Send the actuator commands to robot and go forward one time
    # step (this is where the actual simulation happens)
    robot.set_actuator_commands(tau_2, tau_3)
    robot.step(t=(start_time + (robot.dt * (step + 1))))
    
    # Save image
    robot.snapshot(os.path.join('images', f'image{step:06d}.png'))

At this point, you will have saved a sequence of images:

* `images/image000000.png`
* `images/image000001.png`
* `images/image000002.png`
* ...

You can convert these images to an `mp4` from the command-line as follows, if you have [ffmpeg](https://ffmpeg.org) installed:

```
ffmpeg -framerate 1000 -i images/image%06d.png -vcodec libx264 -pix_fmt yuv420p video_of_results.mp4
```

Note that this must be called from the directory that contains `images`, and if you are on windows you may have to use a backslash instead of a fowardslash as `images\image%06d.png`.